# Imports

In [385]:
import random
import math

# VRPTW

In [386]:
class VRPTW:
    def __init__(self, distance_matrix, time_windows, vehicle_capacity=200, num_vehicles=25, num_ants=25, num_iterations=100):  # Function for initiation with param
        
        # Creating instance variables from params
        self.distance_matrix = distance_matrix
        self.time_windows = time_windows
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.num_ants = num_ants
        self.num_iterations = num_iterations

        self.pheromones = [[1 for _ in range(len(distance_matrix))] for _ in range(len(distance_matrix))]  # Initializing 2D list (matrix); a list of lists
        # Stores the pheromone levels for each route between nodes (customers and the depot); N nodes of distance_matrix results in N x N matrix
        # Rows represent each node
        # Column represents the pheromone level for the route from the row node to the column node
        # for _ in range(len(distance_matrix)), creates outer list with as many rows as there are nodes in the problem (customers + depot); node amount gained from len(distance_matrix)
        # [1 for _ in range(len(distance_matrix))], creates inner list / each row of matrix, initializing each element in row to 1 i.e. all routes are as attractive

        self.best_solution = None

        self.best_cost = float('inf')  # Used to keep track of the best (minimum) cost of a solution found
        # float('inf') is a floating-point value representing positive infinity i.e. the initial value will always be higher than any valid solution later found

        self.best_missed_time_windows = 0  # Used to track how many time windows were missed by the best solution
        self.total_missed_time_windows = 0  # Initialize total missed time windows
        self.penalty_factor = 10 # For not visiting all customers
        self.reward_factor = 1 # For visiting all customers

    def construct_solution(self):  # Generates potential routes for each ant based on the current state of the problem
        solutions = []  # Initializes storage of potential routes / solutions
        best_cost_this_iteration = float('inf')  # Initialize to a large value for this iteration
        best_missed_time_windows_this_iteration = 0  # Track missed time windows for this iteration

        for _ in range(self.num_ants):  # Loop iterates over the number of ants, allowing each ant to construct its own set of routes
            routes = [[] for _ in range(self.num_vehicles)]  # A list of lists, where each inner list represents the route for a specific vehicle
            capacities = [0 for _ in range(self.num_vehicles)]  # A list to keep track of the total load (demand) for each vehicle; initialized to 0
            current_time = [0 for _ in range(self.num_vehicles)]  # A list to track the current time for each vehicle, initialized to 0
            visited_customers = set()  # Initialize visited customers for this solution

            # Start from the depot (node 0)
            for vehicle in range(self.num_vehicles):  # Loop iterates over the number of vehicles, allowing each vehicle to build its own route
                current_location = 0  # Depot i.e. start point for all routes
                while True:
                    # Select the next customer to visit
                    next_customer = self.select_next_customer(current_location, capacities[vehicle], current_time[vehicle], visited_customers)  # Calls method responsible for determining the next customer to visit
                    if next_customer is None:  # If there is no viable next customer found by self.select_next_customer e.g. full capacity for vehicle
                        break  # Stop inner loop i.e. vehicle stops its route construction
                    
                    # Update vehicle state i.e. next customer to visit has successfully been selected
                    routes[vehicle].append(next_customer)  # Append the customer to the vehicle route
                    visited_customers.add(next_customer)  # Mark this customer as visited
                    capacities[vehicle] += customer_demand[next_customer]  # Assuming customer_demand is defined, vehicle's capacity is updated by adding the demand of the selected customer
                    
                    # Update current time considering travel time and time windows
                    current_time[vehicle] += self.distance_matrix[current_location][next_customer]  # Travel time to next customer
                    # Ensure the vehicle adheres to time window constraints
                    current_time[vehicle] = max(current_time[vehicle], self.time_windows[next_customer][0])  # Earliest arrival time
                    current_location = next_customer  # Update current_location to the newly visited customer
                # Return to depot
                if routes[vehicle]:  # Only add if the vehicle has visited customers
                    routes[vehicle].append(0)  # Return to depot


            solutions.append(routes)  # Adds all the complete set of routes for all ants in the solutions variable initialized at the start of the function

            # Debugging - Check if all routes are empty
            # if all(not route for route in routes):
            #    print("All routes are empty for this ant's solution.")

            # Calculate total cost for this solution, including time window penalties
            total_cost = sum(self.calculate_route_cost(route, current_time[vehicle])[0] for vehicle, route in enumerate(routes) if route)
            missed_time_windows = sum(self.calculate_route_cost(route, current_time[vehicle])[1] for vehicle, route in enumerate(routes) if route)
            
            total_customers = set(range(1, len(self.distance_matrix)))  # Set of all customer indices (assuming they start from 1)

            # Penalize or reward based on whether all customers were visited
            if visited_customers == total_customers:
                total_cost * self.reward_factor  # Apply reward
            else:
                unvisited_count = len(total_customers) - len(visited_customers)
                total_cost * unvisited_count * self.penalty_factor  # Apply penalty for unvisited customers

            # print("Total cost for each ant's solution:", total_cost) # Debugging
            # if total_cost == 0: # Debugging
            #    print("Total cost of route is 0 after calculated route cost")
            if total_cost == 0:
                raise ValueError(f"Total cost is 0 for solution: {routes}. Best cost this iteration: {best_cost_this_iteration}")
            
            # Check if this is the best solution so far
            # if len(visited_customers) == total_customers:
            if total_cost < best_cost_this_iteration:
                best_cost_this_iteration = total_cost
                self.best_cost = best_cost_this_iteration
                self.best_solution = routes  # Update to the specific best solution
                best_missed_time_windows_this_iteration = missed_time_windows

        return solutions, best_cost_this_iteration, current_time, best_missed_time_windows_this_iteration  # Function returns the complete list of routes constructed by all ants when called, and the solution with the best / lowest cost

    def select_next_customer(self, current_location, current_capacity, current_time, visited_customers):  # Determines which customer an ant should visit next while constructing its route
        probabilities = []  # Initializes storage of which customers can be visited next, along with a score that represents how likely each customer is to be chosen
        for customer in range(1, len(self.distance_matrix)):  # Iterates over all potential customers, except for 0 which is depot
            
            if current_capacity + customer_demand[customer] <= self.vehicle_capacity and customer not in visited_customers: # Determines if customer can be visited by comparing the vehicles capacity to the current capacity of the route + customer demand + if customer has been visited before
                # Calculate pheromone and heuristic values
                pheromone = self.pheromones[current_location][customer]  # Retrieves the pheromone level between the current location and the next customer from pheromone matrix
                distance = self.distance_matrix[current_location][customer]  # Retrieves the travel distance / time from the current location to the next customer from the distance matrix
                heuristic = 1 / (distance + 1e-6)  # Calculate the heuristic value i.e., the inverse of the distance, for later probability calculation; a small constant (1e-6) is added to avoid division by 0
                probability = pheromone * heuristic  # Calculate the probability / attractiveness of the customer by multiplying the pheromone level by the heuristic value
                probabilities.append((customer, probability))  # Adds valid customer, along with its attractiveness, to the probabilities variable initialized at the start, as a tuple
        
        if not probabilities:  # There are no valid customers so function select_next_customer is to return None, a value recognized by construct_solution function
            return None
        
        # Normalize probabilities, making sure the sum becomes 1 i.e. becomes suitable for probabilistic selection
        total = sum(prob for _, prob in probabilities)  # Sums the values of each probability of valid customers

        probabilities = [(customer, prob / total) for customer, prob in probabilities]  # Redefines the list of customers that can be visited with the new probabilities
        
        # Choose the next customer based on probabilities
        return random.choices([customer for customer, _ in probabilities], weights=[prob for _, prob in probabilities])[0]

    def update_pheromones(self, solutions, current_times):  # Function for updating of the pheromones on each route; takes in argument solutions which are routes generated by the ants during the current iteration
        # Update pheromone levels based on the quality of solutions
        for solution, current_time in zip(solutions, current_times):  # Loops over each solution generated by the ants, and pairs solution with a current time
            total_cost = 0  # Initializes a variable to accumulate the total cost of the routes in the current solution, initialized to 0 as there is no inherent cost
            iteration_missed_time_windows = 0  # Initialize to accumulate missed time windows

            for route_index, route in enumerate(solution):  # Iterates through each route assigned to a vehicle, emurate to get route_index
                if not route:  # Skips any empty routes i.e. vehicles that didn't deliver to any customers / pick any customers
                    continue  # Goes to next route

                # Calculate cost for the route
                route_cost, missed_time_windows = self.calculate_route_cost(route, current_time)  # Calls function calculate_route_cost (defined further down) to calculate the cost of the current route, and unpacks tuple

                iteration_missed_time_windows += missed_time_windows  # Accumulate missed time windows

                if route_cost <= 0: # Ensure route_cost is greater than zero
                    route_cost = 1e-6  # Set a small positive value to avoid division by zero

                total_cost += route_cost  # Adds the calculated cost of the route to the total cost of the solution

                # Update pheromones along the route
                for i in range(len(route) - 1):  # Goes through the segments / paths in the route 
                    self.pheromones[route[i]][route[i + 1]] += 1 / route_cost  # Increase the pheromone level on the path from route[i] to route[i + 1] by the inverse of the route cost
                    # 1 / route_cost is the inverse of the route cost and is used as the increase because the lower the cost, the more pheromone is added, the more desirable the route is for future ants

            self.total_missed_time_windows += iteration_missed_time_windows

            # print(f"Iteration Missed Time Windows: {iteration_missed_time_windows}")

            # Evaporate pheromones i.e. reduce the pheromone levels across all paths; helps prevent convergence to suboptimal solution and encourages exploration
            for i in range(len(self.pheromones)):  # Iterates over all rows
                for j in range(len(self.pheromones[i])):  # Iterates over all columns in the row
                    self.pheromones[i][j] *= 0.95  # Reduces the pheromone on each path by multiplying with a value below 1

    def calculate_route_cost(self, route, current_time):  # Function to calculate the total cost of a specific route taken by a vehicle; takes argument route which is the route to be evaluated
        cost = 0  # Initialize a variable that accumulates the total distance / time of the route as it iterates through the customers; initialized to 0 as there is no inherent cost
        missed_time_windows = 0  # Initialize missed time windows counter
        
        for i in range(len(route) - 1):  # Iterates over the indices of the route list, stopping before the last customer; stops before the last customer as the cost calculation requires looking at pairs of locations (the current and the next)
            cost += self.distance_matrix[route[i]][route[i + 1]]  # Retrieves the distance / travel time between the current location and the next location, and adds it to the cost variable

        # Add time window penalties
        for i in range(len(route)):
            if current_time < self.time_windows[route[i]][0]:  # Early arrival penalty
                cost += self.time_windows[route[i]][0] - current_time  # Add penalty for arriving before the time window
                missed_time_windows += 1  # Increment missed time windows
            elif current_time > self.time_windows[route[i]][1]:  # Late arrival penalty
                cost += current_time - self.time_windows[route[i]][1]  # Add penalty for arriving after the time window
                missed_time_windows += 1  # Increment missed time windows

        return cost, missed_time_windows  # After the for loop has accumulated the total distance / time taken by this route and saved it in cost, the function returns the total cost of the route

    def all_customers_visited(self):
        total_customers = list(range(1, len(self.distance_matrix)))  # Set of all customer indices (assuming they start from 1)
        visited_customers = list()

        # Collect all visited customers from the best solution
        # for route in self.best_solution:
        #    for vehicle_route in route:  # Iterate over each vehicle's route
        #        for customer in vehicle_route:  # Now iterate over each customer in the vehicle's route
        #            visited_customers.add(customer)

        # Collect all visited customers from the best solution
        #for route in self.best_solution:
        #    for vehicle_route in route:  # Iterate over each vehicle's route
        #        visited_customers.update(vehicle_route)  # Add all customers from this route to the visited set

        # Flatten to a single list of numbers
        # flattened_list = [customer for route in self.best_solution for vehicle_route in route for customer in vehicle_route]
        flattened_list =[customer for route in self.best_solution for customer in route]

        sorted_list = sorted(flattened_list)
        sorted_list = sorted_list[self.num_vehicles-1:] # To remove the excess depot

        # Check if all customers are visited
        return visited_customers == total_customers, sorted_list

    def run(self):  # Function to execute the ACO algorithm over a specified number of iterations
        for _ in range(self.num_iterations):  # Runs for the number of desired iterations
            solutions, best_cost_this_iteration, current_times, best_missed_time_windows_this_iteration = self.construct_solution()  # Calls function construct_solution and saves the generated list of routes / solutions for the ants in variable solutions
            
            # Update best solution
            #if best_cost_this_iteration < self.best_cost:  # if this iterations best cost is less than the total best cost; less because we want to minimize cost
            #    # Debugging
            #    #print('Old best_cost', self.best_cost)
            #    
            #    self.best_cost = best_cost_this_iteration  # Sets current iterations cost to total best cost
            #    self.best_solution = solutions  # Sets current solution to to|tal best solution
            #    self.best_missed_time_windows = best_missed_time_windows_this_iteration  # Update best missed time windows
            #    
            #    # Debugging
            #    #print('New best_cost', self.best_cost)
            
            self.update_pheromones(solutions, current_times)  # Calls function update_pheromones, and updates the pheromones on each path using the solutions generated
        
        # Print results at the end of the iterations
        print("Best Cost Found:", self.best_cost)
        print("Best Missed Time Windows Found:", self.best_missed_time_windows)  # Output the best missed time windows
        print ("Best Solution Found: ", self.best_solution)
        print("Best Solution (Routes):")
        for vehicle_index, route in enumerate(self.best_solution):
            print(f"  Vehicle {vehicle_index + 1}: {route}")
        
        # Additional metrics
        print("Total Vehicles Used:", len(self.best_solution))
        print(f"Total Missed Time Windows Across All Iterations: {self.total_missed_time_windows}")
        # total_customers_visited = sum(len(route) for route in self.best_solution)
        # print("Total Customers Visited:", total_customers_visited)

        if self.all_customers_visited()[0]:
            print("All customers have been visited in the best solution.")
        else:
            print(f"Not all customers have been visited in the best solution. {len(self.all_customers_visited()[1])} vs {len(self.distance_matrix)}")
            print("Visited Customers: ", self.all_customers_visited()[1])
            print("All Customers: ", list(range(0, len(self.distance_matrix))))

# c101 constraints

In [387]:
# Handle data from c101
x_coords = [
    40, 45, 45, 42, 42, 42, 40, 40, 38, 38, 35, 35, 25, 22, 22, 20,
    20, 18, 15, 15, 30, 30, 28, 28, 25, 25, 25, 23, 23, 20, 20, 10,
    10, 8, 8, 5, 5, 2, 0, 0, 35, 35, 33, 33, 32, 30, 30, 30, 28,
    28, 26, 25, 25, 44, 42, 42, 40, 40, 38, 38, 35, 50, 50, 50, 48,
    48, 47, 47, 45, 45, 95, 95, 53, 92, 53, 45, 90, 88, 88, 87, 85,
    85, 75, 72, 70, 68, 66, 65, 65, 63, 60, 60, 67, 65, 65, 62, 60,
    60, 58, 55, 55
]

y_coords = [
    50, 68, 70, 66, 68, 65, 69, 66, 68, 70, 66, 69, 85, 75, 85, 80,
    85, 75, 75, 80, 50, 52, 52, 55, 50, 52, 55, 52, 55, 50, 55, 35,
    40, 40, 45, 35, 45, 40, 40, 45, 30, 32, 32, 35, 30, 30, 32, 35,
    30, 35, 32, 30, 35, 5, 10, 15, 5, 15, 5, 15, 5, 30, 35, 40, 30,
    40, 35, 40, 30, 35, 30, 35, 30, 30, 35, 65, 35, 30, 35, 30, 25,
    35, 55, 55, 58, 60, 55, 55, 60, 58, 55, 60, 85, 85, 82, 80, 80,
    85, 75, 80, 85
]

# Combine x and y coordinates into a list of tuples
coordinates = list(zip(x_coords, y_coords))

# Initialize the distance matrix
num_locations = len(coordinates)
distance_matrix = [[0 for _ in range(num_locations)] for _ in range(num_locations)]

# Populate the distance matrix
for i in range(num_locations):
    for j in range(num_locations):
        if i != j:  # No need to calculate distance to itself
            x1, y1 = coordinates[i]
            x2, y2 = coordinates[j]
            distance_matrix[i][j] = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Define the ready and due times
ready_times = [
    0, 912, 825, 65, 727, 15, 621, 170, 255, 534, 357, 448, 652,
    30, 567, 384, 475, 99, 179, 278, 10, 914, 812, 732, 65, 169,
    622, 261, 546, 358, 449, 200, 31, 87, 751, 283, 665, 383, 479,
    567, 264, 166, 68, 16, 359, 541, 448, 1054, 632, 1001, 815,
    725, 912, 286, 186, 95, 385, 35, 471, 651, 562, 531, 262,
    171, 632, 76, 826, 12, 734, 916, 387, 293, 450, 478, 353,
    997, 203, 574, 109, 668, 769, 47, 369, 265, 458, 555, 173,
    85, 645, 737, 20, 836, 368, 475, 285, 196, 95, 561, 30, 743,
    647
]

due_times = [
    1236, 967, 870, 146, 782, 67, 702, 225, 324, 605, 410, 505,
    721, 92, 620, 429, 528, 148, 254, 345, 73, 965, 883, 777, 144,
    224, 701, 316, 593, 405, 504, 237, 100, 158, 816, 344, 716,
    434, 522, 624, 321, 235, 149, 80, 412, 600, 509, 1127, 693,
    1066, 880, 786, 969, 347, 257, 158, 436, 87, 534, 740, 629,
    610, 317, 218, 693, 129, 875, 77, 777, 969, 456, 360, 505,
    551, 412, 1068, 260, 643, 170, 731, 820, 124, 420, 338, 523,
    612, 238, 144, 708, 802, 84, 889, 441, 518, 336, 239, 156,
    622, 84, 820, 726
]

# Combine ready and due times into a list of tuples
time_windows = list(zip(ready_times, due_times))

# Ensure the lengths match with the number of locations
num_locations = len(time_windows)  # Should match the distance_matrix length

customer_demand = [
    0, 10, 30, 10, 10, 10, 20, 20, 20, 10, 10, 10, 20, 30, 10, 40, 40, 20, 20, 10, 10, 20, 20, 10, 10, 40, 10, 10, 20, 10, 10, 20, 30, 40, 20, 10, 10, 20, 30, 20, 10, 10, 20, 10, 10, 10, 30, 10, 10, 10, 10, 10, 10, 20, 40, 10, 30, 40, 30, 10, 20, 10, 20, 50, 10, 10, 10, 10, 10, 10, 30, 20, 10, 10, 50, 20, 10, 10, 20, 10, 10, 30, 20, 10, 20, 30, 10, 20, 30, 10, 10, 10, 20, 40, 10, 30, 10, 30, 20, 10, 20 
]

service_time = [
    0, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90
]

# Run ACO

In [388]:
aco = VRPTW(distance_matrix, time_windows, num_vehicles=25, num_ants=25, num_iterations=100)
aco.run()

Best Cost Found: 40264.54542403558
Best Missed Time Windows Found: 0
Best Solution Found:  [[5, 3, 4, 6, 9, 8, 11, 10, 7, 58, 56, 53, 0], [19, 18, 37, 38, 39, 29, 27, 28, 26, 25, 24, 0], [45, 48, 50, 51, 52, 46, 42, 41, 40, 43, 44, 33, 31, 0], [87, 86, 89, 88, 90, 57, 55, 59, 60, 81, 79, 0], [62, 66, 69, 67, 65, 63, 74, 15, 0], [16, 14, 12, 99, 98, 94, 96, 95, 97, 100, 0], [17, 13, 77, 73, 70, 71, 76, 78, 64, 61, 72, 68, 20, 0], [21, 22, 30, 75, 1, 2, 91, 92, 93, 84, 0], [83, 82, 85, 36, 34, 32, 35, 80, 54, 49, 47, 0], [23, 0], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Best Solution (Routes):
  Vehicle 1: [5, 3, 4, 6, 9, 8, 11, 10, 7, 58, 56, 53, 0]
  Vehicle 2: [19, 18, 37, 38, 39, 29, 27, 28, 26, 25, 24, 0]
  Vehicle 3: [45, 48, 50, 51, 52, 46, 42, 41, 40, 43, 44, 33, 31, 0]
  Vehicle 4: [87, 86, 89, 88, 90, 57, 55, 59, 60, 81, 79, 0]
  Vehicle 5: [62, 66, 69, 67, 65, 63, 74, 15, 0]
  Vehicle 6: [16, 14, 12, 99, 98, 94, 96, 95, 97, 100, 0]
  Vehicle 7: [17, 13, 77, 